In [1]:
# import libraries
from pathlib import Path
import requests
import math
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# assign a path for data files
DATA_DIR   = "../../data/raw/"
RESULT_DIR = "../../data/processed/"

## Household Income values

In [2]:
# Read in pandas and show the first 5rows
df_household_income = pd.read_csv(f"{DATA_DIR}Income_variable.csv", skiprows=3)
df_household_income.head()

,GeoFips,GeoName,LineCode,Description,2020
0,00000,United States,NaN,Real dollar statistics,NaN
1,00000,United States,1.0,Real GDP (millions of chained 2012 dollars) 1/,18509143.0
2,00000,United States,2.0,Real personal income (millions of constant (...,17628679.9
3,00000,United States,3.0,Real PCE (millions of constant (2012) dollar...,12629896.7
4,00000,United States,NaN,Current dollar statistics (millions of dollars),NaN


In [3]:
# Drop the row which has the `LineCode` column as NaN
df_household_income.dropna(axis=0, inplace=True)
df_household_income.reset_index(drop=True, inplace=True)

In [4]:
# convert all the `2020` str data to float
for i in range(0,len(df_household_income['2020'])):
    if df_household_income['2020'][i] != "(NA)":
        df_household_income['2020'][i] = np.float64(df_household_income['2020'][i])
    else:
        df_household_income['2020'][i] = np.nan

# rename the column name without the whitespace
df_household_income.rename(columns=lambda x: x.strip(), inplace=True)

<ipython-input-4-158a971760a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_household_income['2020'][i] = np.float64(df_household_income['2020'][i])
<ipython-input-4-158a971760a4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_household_income['2020'][i] = np.nan


In [5]:
# make the table format tidy with data names for column
df_household_income = df_household_income.pivot(index="GeoName", columns="Description", values="2020")

In [6]:
# We convert the GeoNmae to abbreviation
## get the abbreviation from abbreviation data
abbrevaition = pd.read_csv(f"{DATA_DIR}state_abbreviation.csv", index_col="full_name")

## get the index names in all caps 
df_household_income.rename(index=lambda x: x.upper(), inplace=True)

## drop "UNITED STATES" because it is irrelevant for our election prediction
df_household_income.drop("UNITED STATES", inplace=True)

## get the new index in a list for renaming the indecies
old_index = list(df_household_income.index)
new_index = []
for index in old_index:
    new_index.append(abbrevaition.loc[index].values[0])

## reindex using the `new_index`
df_household_income.index = new_index

## rename index name
df_household_income.columns.name = "state"

# show first 5 rows
df_household_income.head()

state,Disposable personal income,Gross domestic product (GDP),Implicit regional price deflator 10/,Per capita disposable personal income 7/,Per capita personal consumption expenditures (PCE) 8/,Per capita personal income 6/,Personal consumption expenditures,Personal income,Real GDP (millions of chained 2012 dollars) 1/,Real PCE (millions of constant (2012) dollars) 3/,Real per capita PCE 5/,Real per capita personal income 4/,Real personal income (millions of constant (2012) dollars) 2/,Regional price parities (RPPs) 9/,Total employment (number of jobs)
AL,211949.4,230892.1,99.27,42181.0,35244.0,46179.0,177093.9,232040.3,199880.8,177776.9,36122.0,46963.0,231128.3,89.252,2671253.0
AK,42437.8,50475.2,114.827,57940.0,48921.0,62756.0,35831.5,45965.1,50705.2,31034.2,42445.0,55470.0,40557.5,103.239,430824.0
AZ,339835.8,382072.3,110.192,47344.0,40237.0,52327.0,288818.8,375601.3,327178.0,260537.2,35106.0,45193.0,335393.2,99.071,3920036.0
AR,129873.4,133969.1,99.191,43115.0,34786.0,47154.0,104782.9,142038.5,117268.2,105340.8,34760.0,47765.0,144752.4,89.181,1640442.0
CA,2403799.4,3020173.4,122.769,60856.0,46802.0,70647.0,1848651.7,2790523.5,2667220.9,1495470.9,37987.0,57347.0,2257635.2,110.38,23155486.0


In [7]:
# rename the columns
old_col = list(df_household_income.columns)
new_col = ["disposable_personal_income", "GDP", "IRPD", "disposable_personal_income_capita", "PCE_capita", "personal_income_capita", "personal_consumption_expenditure", "personal_income",
            "real_GDP_2012", "real_PCE_2012", "real_per_capita_PCE", "real_per_capita_personal_income", "real_personal_income", "RPPs", "total_employment"]
col_dict = dict(zip(old_col, new_col))
df_household_income.rename(columns=col_dict, inplace=True)

# show first 5 rows
df_household_income.head()

state,disposable_personal_income,GDP,IRPD,disposable_personal_income_capita,PCE_capita,personal_income_capita,personal_consumption_expenditure,personal_income,real_GDP_2012,real_PCE_2012,real_per_capita_PCE,real_per_capita_personal_income,real_personal_income,RPPs,total_employment
AL,211949.4,230892.1,99.27,42181.0,35244.0,46179.0,177093.9,232040.3,199880.8,177776.9,36122.0,46963.0,231128.3,89.252,2671253.0
AK,42437.8,50475.2,114.827,57940.0,48921.0,62756.0,35831.5,45965.1,50705.2,31034.2,42445.0,55470.0,40557.5,103.239,430824.0
AZ,339835.8,382072.3,110.192,47344.0,40237.0,52327.0,288818.8,375601.3,327178.0,260537.2,35106.0,45193.0,335393.2,99.071,3920036.0
AR,129873.4,133969.1,99.191,43115.0,34786.0,47154.0,104782.9,142038.5,117268.2,105340.8,34760.0,47765.0,144752.4,89.181,1640442.0
CA,2403799.4,3020173.4,122.769,60856.0,46802.0,70647.0,1848651.7,2790523.5,2667220.9,1495470.9,37987.0,57347.0,2257635.2,110.38,23155486.0


In [22]:
# store in the `processed` data file
df_household_income.to_csv(f'{RESULT_DIR}household_income.csv')

## Life Expectancy

In [9]:
# Load csv and show first 5 rows
df_life_expectancy_with_url = pd.read_csv(f"{DATA_DIR}life_expectancy.csv")
df_life_expectancy_with_url.head()

,YEAR,STATE,RATE,URL
0,2020,AL,73.2,/nchs/pressroom/states/alabama/al.htm
1,2020,AK,76.6,/nchs/pressroom/states/alaska/ak.htm
2,2020,AZ,76.3,/nchs/pressroom/states/arizona/az.htm
3,2020,AR,73.8,/nchs/pressroom/states/arkansas/ar.htm
4,2020,CA,79.0,/nchs/pressroom/states/california/ca.htm


In [10]:
# scrape the data within the URL link for each states
base_url = "https://www.cdc.gov"
final_df = []
for url in df_life_expectancy_with_url["URL"]:
    state_health_cont = requests.get(f'{base_url}{url}')
    state_health_soup = BeautifulSoup(state_health_cont.content, 'html.parser')
    state_health_html = state_health_soup.find("table", class_='table table-bordered fs08 table-striped bg-gray-l3')

    # extract data from the table 
    data = []
    for row in state_health_html.find_all("tr"):
        cells = row.find_all("td")
        if cells:
            data.append([cell.text for cell in cells])

    # Create a dataframe from the extracted data
    temp_df   = pd.DataFrame(data).T
    temp_dict = dict(zip(list(temp_df.iloc[0]), list(temp_df.iloc[1])))
    df_state  = pd.DataFrame([temp_dict])

    # append to the overall dataframe list
    final_df.append(df_state)

# concat the bundle of retrieved dataframe and formulate one dataframe
df_USA_health = pd.concat(final_df)

# delete the unnecessary columns generated through concat
drop_col = [df_USA_health.columns[i] for i in range(2)]+ [df_USA_health.columns[11]]
df_USA_health.drop(drop_col, axis=1, inplace=True)

# show the first 5 rows in dataframe
df_USA_health.head()

,Infant Mortality Rate,Life Expectancy (at Birth),Marriage Rate,Divorce Rate,Leading Cause of Death,Drug Overdose Death Rate,Firearm Death Rate,Homicide Rate,"COVID-19 Death Rate (Q4, 2021)",Firearm Injury Death Rate,"COVID-19 Death Rate (Q1, 2022)"
0,"7.2 (infant deaths per 1,000 live births)",73.2 years (2020),"7.2 (marriages per 1,000)","3.7 (divorces per 1,000)",Heart Disease,"22.3 (per 100,000)¹","23.6 (per 100,000)¹","14.2 (per 100,000)¹","89.3 (per 100,000)²",NaN,NaN
0,"5.0 (infant deaths per 1,000 live births)",76.6 years (2020),"5.7 (marriages per 1,000)","3.2 (divorces per 1,000)",Cancer,"22.0 (per 100,000)¹",NaN,"7.3 (per 100,000)¹","197.9 (per 100,000)²","23.5 (per 100,000)¹",NaN
0,"5.0 (infant deaths per 1,000 live births)",76.3 years (2020),"4.9 (marriages per 1,000)","2.9 (divorces per 1,000)",Heart Disease,"35.8 (per 100,000)¹",NaN,"7.5 (per 100,000)¹","196.8 (per 100,000)²","16.7 (per 100,000)¹",NaN
0,"7.3 (infant deaths per 1,000 live births)",73.8 years (2020),"7.8 (marriages per 1,000)","3.6 (divorces per 1,000)",Heart Disease,"19.1 (per 100,000)¹",NaN,"13.0 (per 100,000)¹","91.9 (per 100,000)²","22.6 (per 100,000)¹",NaN
0,"3.7 (infant deaths per 1,000 live births)",79.0 years (2020),"3.2 (marriages per 1,000)",n/a,Heart Disease,"21.8 (per 100,000)¹",NaN,"6.1 (per 100,000)¹","62.1 (per 100,000)²","8.5 (per 100,000)¹",NaN


In [11]:
# use this code block to extract only the numeric values for rellevant columns 
for i in range(len(df_USA_health.columns)):
    if i != 4:
        # we extract the values by columns
        vals = df_USA_health.iloc[:, i]
        for j in range(150):
            val = vals.iloc[j]

            # if the data type is a float, the cell has a float nan value
            if type(val) != float: 
                # if it is NA values we will denote as numpy NA values
                if val == "n/a" or "N/A" in val:
                    df_USA_health.iloc[j][i] = np.NaN
                else:
                    df_USA_health.iloc[j][i] = np.float64(re.findall(r"\d+\.\d", val)[0])
            else:
                df_USA_health.iloc[j][i] = np.NaN
    # Avoid leading cause of death column since it is not numerical
    else:
        continue

# show first 5 rows of dataframe
df_USA_health.head()

,Infant Mortality Rate,Life Expectancy (at Birth),Marriage Rate,Divorce Rate,Leading Cause of Death,Drug Overdose Death Rate,Firearm Death Rate,Homicide Rate,"COVID-19 Death Rate (Q4, 2021)",Firearm Injury Death Rate,"COVID-19 Death Rate (Q1, 2022)"
0,7.2,73.2,7.2,3.7,Heart Disease,22.3,23.6,14.2,89.3,NaN,NaN
0,5.0,76.6,5.7,3.2,Cancer,22.0,NaN,7.3,197.9,23.5,NaN
0,5.0,76.3,4.9,2.9,Heart Disease,35.8,NaN,7.5,196.8,16.7,NaN
0,7.3,73.8,7.8,3.6,Heart Disease,19.1,NaN,13.0,91.9,22.6,NaN
0,3.7,79.0,3.2,NaN,Heart Disease,21.8,NaN,6.1,62.1,8.5,NaN


In [20]:
# make a final dataframe by concatting the two dataframe
df_USA_health.reset_index(inplace=True)
df_life_expectancy_with_url.reset_index(inplace=True)
df_life_expectancy = pd.concat([df_life_expectancy_with_url, df_USA_health], axis=1)

# drop the unnecessary columns generated in concat
df_life_expectancy.drop(columns=['URL', 'index'], inplace=True)

# rename `STATE` column for standardizing the index
df_life_expectancy.rename(columns={"STATE":"state"}, inplace=True)

# show the first 5 rows of the final dataframe
df_life_expectancy.head()

,YEAR,state,RATE,Infant Mortality Rate,Life Expectancy (at Birth),Marriage Rate,Divorce Rate,Leading Cause of Death,Drug Overdose Death Rate,Firearm Death Rate,Homicide Rate,"COVID-19 Death Rate (Q4, 2021)",Firearm Injury Death Rate,"COVID-19 Death Rate (Q1, 2022)"
0,2020,AL,73.2,7.2,73.2,7.2,3.7,Heart Disease,22.3,23.6,14.2,89.3,NaN,NaN
1,2020,AK,76.6,5.0,76.6,5.7,3.2,Cancer,22.0,NaN,7.3,197.9,23.5,NaN
2,2020,AZ,76.3,5.0,76.3,4.9,2.9,Heart Disease,35.8,NaN,7.5,196.8,16.7,NaN
3,2020,AR,73.8,7.3,73.8,7.8,3.6,Heart Disease,19.1,NaN,13.0,91.9,22.6,NaN
4,2020,CA,79.0,3.7,79.0,3.2,NaN,Heart Disease,21.8,NaN,6.1,62.1,8.5,NaN


In [21]:
# store in the `processed` data file
df_life_expectancy.to_csv(f'{RESULT_DIR}life_expectancy.csv', index=False)

## Education Level

In [17]:
# load the csv for education level
df_education = pd.read_csv(f"{DATA_DIR}education_data.csv")

# show the first 5 rows
df_education.head()

,fips,state,densityMi,pop2022,pop2021,pop2020,pop2019,pop2010,growthRate,growth,growthSince2010,PercentHighSchoolOrHigher,PercentBachelorsOrHigher
0,30,Montana,7.57964,1103187,1093706,1084225,1074744,989415,0.00867,9481,0.11499,94.0,33.1
1,56,Wyoming,5.96845,579495,578173,576851,575524,563626,0.00229,1322,0.02816,93.6,28.2
2,50,Vermont,70.14701,646545,644811,643077,641347,625741,0.00269,1734,0.03325,93.5,39.7
3,27,Minnesota,72.67645,5787008,5746751,5706494,5666238,5303925,0.00701,40257,0.09108,93.4,36.8
4,33,New Hampshire,155.22629,1389741,1383635,1377529,1371424,1316470,0.00441,6106,0.05566,93.3,37.6


In [18]:
# change the state name to geo code and set that as an index
## for the `state` column, we change the value to all capital letters
for state in df_education.iloc[:, 1].values:
    df_education.loc[df_education['state']==state, "state"] = state.upper()

## set the state column as index
df_education.set_index('state', inplace=True)

## get the new index in a list for renaming the indecies
old_index = list(df_education.index)
new_index = []
for index in old_index:
    new_index.append(abbrevaition.loc[index].values[0])

## reindex using the `new_index`
df_education.index = new_index

# show first 5 rows
df_education.head()

,fips,densityMi,pop2022,pop2021,pop2020,pop2019,pop2010,growthRate,growth,growthSince2010,PercentHighSchoolOrHigher,PercentBachelorsOrHigher
MT,30,7.57964,1103187,1093706,1084225,1074744,989415,0.00867,9481,0.11499,94.0,33.1
WY,56,5.96845,579495,578173,576851,575524,563626,0.00229,1322,0.02816,93.6,28.2
VT,50,70.14701,646545,644811,643077,641347,625741,0.00269,1734,0.03325,93.5,39.7
MN,27,72.67645,5787008,5746751,5706494,5666238,5303925,0.00701,40257,0.09108,93.4,36.8
NH,33,155.22629,1389741,1383635,1377529,1371424,1316470,0.00441,6106,0.05566,93.3,37.6


In [19]:
# store csv into the `processed` file
df_education.to_csv(f'{RESULT_DIR}education.csv')